In [14]:
# Colab and system related
import os
import time
from google.colab import drive
import sys
drive.mount('/content/drive')

#Nifty
if os.path.isdir('nifty') == False:
    !git clone https://gitlab.mpcdf.mpg.de/ift/nifty.git
sys.path.append('/content/nifty')
import nifty6 as ift
sys.path.append('/content/drive/My Drive/masterthesis/autoencoder')
from operators.tensorflow_operator import TensorFlowOperator

# Tensorflow
import tensorflow as tf

# Plotting
from matplotlib import pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi']= 150

# Numerics
import random
import numpy as np
from scipy.stats import multivariate_normal
from sklearn.neighbors import KernelDensity # required for density estimation of latent-space representation
from scipy.spatial import distance # required for Mahalanobis-Distance
!pip install uncertainties
from uncertainties import ufloat
from uncertainties import unumpy
from scipy.optimize import minimize
from scipy.optimize import Bounds
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from tensorflow.keras import Input

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# Load MNIST Dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [16]:
x_train = x_train.reshape((-1, 784))
x_test = x_test.reshape((-1, 784))
input_shape = (784,)
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value.
x_train /= 255
x_test /= 255


In [17]:
Input = tf.keras.layers.Input(shape=784)
h1 = tf.keras.layers.Dense(512, activation='elu')(Input)
h2 = tf.keras.layers.Dense(256, activation='elu')(h1)
h3 = tf.keras.layers.Dense(128, activation='elu')(h2)
h4 = tf.keras.layers.Dense(10, activation='linear')(h3)
encoded = tf.keras.activations.softmax(h4)
model = tf.keras.Model(Input, encoded)

In [18]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.fit(x_train,y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 8s 141us/sample - loss: 0.2324 - acc: 0.9291
Epoch 2/5
60000/60000 [==============================] - 9s 143us/sample - loss: 0.1170 - acc: 0.9634
Epoch 3/5
60000/60000 [==============================] - 9s 143us/sample - loss: 0.0883 - acc: 0.9726
Epoch 4/5
60000/60000 [==============================] - 9s 142us/sample - loss: 0.0706 - acc: 0.9784
Epoch 5/5
60000/60000 [==============================] - 9s 143us/sample - loss: 0.0601 - acc: 0.9816


In [19]:
model.evaluate(x_test, y_test)

[0.09406351856330876, 0.9772]

In [20]:
model.save('drive/My Drive/masterthesis/autoencoder/trained_models/Classifier/', save_format='tf')

INFO:tensorflow:Assets written to: drive/My Drive/masterthesis/autoencoder/trained_models/Classifier/assets


In [21]:
print(x_test[10, :].shape)
print(x_test.shape)
#a = (loss_fn(y_test[10], model.predict(np.array([x_test[10, :]]))))
print(y_test[20])
print(tf.keras.backend.get_value(model.predict(np.array([x_test[20, :]]))))
print(model.layers[-2].output)

(784,)
(10000, 784)
9
[[4.3522380e-08 7.9188595e-07 5.5864842e-09 1.5496007e-04 3.3656196e-03
  8.6686799e-05 2.4216434e-10 3.6300815e-04 8.6525033e-06 9.9602026e-01]]
Tensor("dense_11/BiasAdd:0", shape=(?, 10), dtype=float32)
